<a href="https://colab.research.google.com/github/avner8943/bootcamp/blob/master/radvess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
from zipfile import ZipFile as zp
import cv2
import os
import numpy as np
import shutil

for zip_f in glob('*.zip'):

    actor_dir = zip_f[:-4]

    if os.path.isdir(actor_dir):
      shutil.rmtree(actor_dir)

    os.mkdir(actor_dir)

    with zp('/content/'+zip_f) as myzip:

      n_vids = 1

      for f in myzip.infolist():
        
        vid_dir = actor_dir + '/vid_' +str(n_vids)

        print(vid_dir)

        os.mkdir(vid_dir )


        ifile = myzip.extract(f)
        
        vidcap = cv2.VideoCapture(ifile)
        
        fps = vidcap.get(cv2.CAP_PROP_FPS)

        step = np.round(fps/16)

        success,image = vidcap.read()
        count = 0
        while success:
          if count % step == 0:
              
              cv2.imwrite(vid_dir + "/frame_{d}.jpg".format(d = count), image)     # save frame as JPEG file      
          success,image = vidcap.read()
          count += 1
        os.remove(ifile)
        n_vids += 1

Actor_01/vid_1
Actor_01/vid_2
Actor_01/vid_3


In [48]:
import shutil
for f in glob('*.jpg'):

  os.remove(f)